# CNN Model Building

Importing The Model Building Libraries

In [2]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 312 kB 24.0 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires keras<2.10.0,>=2.9.0rc0, but you have keras 2.2.4 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 454.3 MB 14 kB/s 
     |████████████████████████████████| 1.2 MB 49.9 MB/s 
     |████████████████████████████████| 462 kB 56.3 MB/s 
     |████████████████████████████████| 14.8 MB 50.3 MB/s 
     |███████████████████████████

In [23]:
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout 
from keras_preprocessing.image import ImageDataGenerator


Initializing The Model

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [12]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile('Fruites Data.zip','r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [32]:
import os
filenames = os.listdir('/content/Fruites Data/trainset')

In [33]:
x_train = train_datagen.flow_from_directory(
    '/content/Fruites Data/trainset',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test_datagen.flow_from_directory(
    '/content/Fruites Data/trainset',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 2626 images belonging to 5 classes.
Found 2626 images belonging to 5 classes.


In [34]:
print(x_train.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [35]:
print(x_test.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [36]:
from collections import Counter as c
c(x_train .labels)


Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

Adding CNN Layers

In [37]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32,(3,3),input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))

# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())


Adding Dense Layers

In [ ]:
# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2

Configure the Learning Process

In [38]:
classifier.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

Train The Model

In [40]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Epoch 1/20
526/526 [==============================] - 38s 70ms/step - loss: 0.1588 - accuracy: 0.9391 - val_loss: 2.5251e-04 - val_accuracy: 1.0000
Epoch 2/20
526/526 [==============================] - 31s 59ms/step - loss: 3.6180e-04 - accuracy: 1.0000 - val_loss: 1.0743e-04 - val_accuracy: 1.0000
Epoch 3/20
526/526 [==============================] - 32s 60ms/step - loss: 0.0432 - accuracy: 0.9924 - val_loss: 2.9558e-04 - val_accuracy: 1.0000
Epoch 4/20
526/526 [==============================] - 31s 60ms/step - loss: 4.8542e-04 - accuracy: 1.0000 - val_loss: 2.3154e-05 - val_accuracy: 1.0000
Epoch 5/20
526/526 [==============================] - 32s 61ms/step - loss: 7.1010e-05 - accuracy: 1.0000 - val_loss: 1.0690e-05 - val_accuracy: 1.0000
Epoch 6/20
526/526 [==============================] - 33s 62ms/step - loss: 0.0101 - accuracy: 0.9954 - val_loss: 0.2096 - val_accuracy: 0.9349
Epoch 7/20
526/526 [==============================] - 32s 61ms/step - loss: 0.0283 - accuracy: 0.9935 - 

 Save the model

In [41]:

classifier.save('Fruites.h5')

In [44]:
!tar -zcvf image-classification-model_new.tgz Fruites.h5

Fruites.h5


In [45]:
ls -1

'Fruites Data'/
'Fruites Data.zip'
Fruites.h5
image-classification-model_new.tgz
sample_data/


Test The Model

In [47]:
from tensorflow.keras.models import load_model
from keras_preprocessing import image
model = load_model("Fruites.h5") 

In [49]:
#loading of the image
img = image.load_img('/content/Nagpur_orange_article.jpg',target_size= (64,64))
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict_classes(x)#predicting the classes
pred

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([2])

In [50]:
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
result=str(index[pred[0]])
result

'ORANGE'